In [13]:
import pandas as pd
import ast
import numpy as np
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Dataset

In [17]:
training_path = "../data/train.csv"
valid_path = "../data/"
train_df = pd.read_csv(training_path)
train_df.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [28]:

train_df['bbox'] = train_df['bbox'].apply(lambda x: ast.literal_eval(x))
print(type(train_df['bbox'][0]))


<class 'list'>


In [29]:
bbox_list = np.array(list(train_df['bbox']))

for i,dimension in enumerate(['x','y','w','h']):
    train_df[dimension] = bbox_list[:,i]
train_df.head()

,image_id,width,height,bbox,source,x,y,w,h
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1,834.0,222.0,56.0,36.0
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1,226.0,548.0,130.0,58.0
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1,377.0,504.0,74.0,160.0
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1,834.0,95.0,109.0,107.0
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1,26.0,144.0,124.0,117.0


In [20]:
class WheatDataset(Dataset):
    def __init__(self, dataframe, image_dir, transforms = None):
        super().__init__()
        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
        
    def __getitem__(self,idx):
        image_id = self.image_ids[idx]
        record = self.df[self.df['image_id'] == image_id]
        image_name = image_id + 'jpg'
        img = Image.open(image_dir/image_name).convert("RGB")
        img = T.ToTensor()(img)

        boxes = records[['x','y','w','h']].values
        boxes[:,2] = boxes[:,0]+boxes[:,2]
        boxes[:,3] = boxes[:,1]+boxes[:,3]
        boxes = torch.tensor(boxes, dtype=torch.int64)
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([idx])

        return img, target, image_id

    def __len__():
        return self.image_ids.shape[0]


In [5]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\salon/.cache\torch\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [9]:
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [24]:
train_dataset = WheatDataset(train_df,training_path)
type(train_dataset)

train_data_loader = DataLoader(train_dataset, batch_size = 16, shuffle = False, num_workers = 4)

In [26]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [28]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
num_epochs = 2


In [29]:
for epoch in range(num_epochs):
    for images, targets, image_ids in train_data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item() 

BrokenPipeError: [Errno 32] Broken pipe